In [1]:
year = 1962

In [9]:

import os

class B:

    BLUE = "#00B1E4"
    LIGHT_BLUE = "rgba(0, 177, 228, 0.5)"
    RED = "#FF6384"
    LIGHT_RED = "rgba(255, 99, 132, 0.5)"
    GREEN = "#00B275"
    LIGHT_GREEN = "rgb(0, 178, 117, 0.5)"

    def __init__(
        self,
        data,
        title="",
        percent_change=None,
        progress=None,
        color=None,
        points=None,
        chart_type=None,
    ):
        self.data = data
        self.title = title
        self.blox_type = "numeric"
        if isinstance(self.data, list):
            self.blox_type = "list"
        self.percent_change = percent_change
        self.progress = int(progress) if progress is not None else None
        self.color = color
        self.points = points
        self.chart_type = chart_type

        # position when displayed as a list
        self.position = None

    def _repr_html_(self):
        if self.blox_type == "list":
            bloxs = ""
            for i, b in enumerate(self.data):
                if isinstance(b, B):
                    b.position = i
                    bloxs += b._repr_html_()

            return f"""<iframe><html><body><div style="display: flex; background: #fff">{bloxs}</div></body></html></iframe>"""

        percent_change_html = ""
        if self.percent_change is not None:
            if self.percent_change > 0:
                percent_change_html = f"""
                <span style="font-size: 1.3em; color: {self.GREEN}; font-family: monospace;"> + {self.percent_change}%</span>
                """
            else:
                percent_change_html = f"""
                <span style="font-size: 1.3em; color: {self.RED}; font-family: monospace;"> {self.percent_change}%</span>
                """
        title_html = ""
        if self.title != "":
            title_html = f"""<span style="font-size: 2em; color: gray; display: block; padding-top: 20px; font-family: monospace; line-height: 1.3em;">{self.title}</span>"""

        progress_html = ""
        if self.progress is not None:
            color = self.BLUE
            if self.color is not None:
                if self.color == "red":
                    color = self.RED
                elif self.color == "blue":
                    color = self.BLUE
                elif self.color == "green":
                    color = self.GREEN
                else:
                    color = self.color

            progress_html = f"""
            <div style="background-color:#f1f1f1; margin: 10px">
                <div style="background-color:{color}; height:24px;width:{self.progress}%"></div>
            </div>
            """
        data_str = ""
        if isinstance(self.data, str):
            data_str = self.data
        else:
            data_str = f"{self.data:,}"

        script_html, chart_html = self.construct_chart_html()

        padding_bottom = "0px" if chart_html != "" else "40px"
        return f"""
<div style="text-align: center; width: 34%; border: 1px solid #e5e5e5; margin: 10px; padding-top: 40px; padding-bottom: {padding_bottom}; background: white; border-radius:5px">
  <span style="font-size: 4em; color: #5a5a5a; font-family: monospace; ">{data_str}</span>
  {percent_change_html}
  {title_html}
  {progress_html}
  {chart_html}
</div>

  {script_html}
  """

    def construct_chart_html(self):
        script_html, chart_html = "", ""

        if self.points is None:
            return script_html, chart_html

        title = (
            self.title
            if self.title is not None
            else f"chart-{len(self.points)}-{self.points[0]}"
        )

        position = self.position if self.position is not None else 0
        chartId = f"id-{title}-{position}"

        chart_html = (
            f"""<canvas id="{chartId}" style="width: 100%; border: 0px"></canvas>"""
        )

        border_color = self.BLUE
        fill_color = self.LIGHT_BLUE
        if self.color is not None:
            if self.color == "red":
                border_color = self.RED
                fill_color = self.LIGHT_RED
            elif self.color == "blue":
                border_color = self.BLUE
                fill_color = self.LIGHT_BLUE
            elif self.color == "green":
                border_color = self.GREEN
                fill_color = self.LIGHT_GREEN
            else:
                border_color = self.color

        points_str = ",".join([str(i) for i in self.points])

        stepped_html = ""
        if self.chart_type == "stepped":
            stepped_html = "stepped: 'middle',"

        script_html = ""
        if position == 0:
            script_html = f"""
<script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/3.7.1/chart.min.js" integrity="sha512-QSkVNOCYLtj73J4hbmVoOV6KVZuMluZlioC+trLpewV8qMjsWqlIQvkn1KGX2StWvPMdWGBqim1xlC8krl1EKQ==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
            
        """
        # need to figure out the way for offline serving ... ????
        # <script src="{self.path}/chart.3.7.1.min.js"></script>
        script_html += f"""
<script>
  window.addEventListener('load', fn);
  
  function fn(){{
    
    const labels{position} = Array({len(self.points)}).fill('');

    const data{position} = {{
      labels: labels{position},
      datasets: [{{
        label: 'T{title}',
        backgroundColor: '{fill_color}',
        borderColor: '{border_color}',
        data: [{points_str}],
        fill: true,
        {stepped_html}
      }}]
    }};
  }}
  
  """
        script_html += f"""
  const config{position} = {{
    type: '"""
        script_html += "bar" if self.chart_type == "bar" else "line"
        script_html += f"""',
    data: data{position}"""
        script_html += """,
    options: { 
     elements: {
       point: {
         radius: 0,
       }
     },
     plugins: {
       legend: {
         display: false
       }
     },
     layout: {
       autoPadding: false
     },
     scales: {
       x: {
         display: false
       },
       y: {
         display: false
         """
        script_html += """
       }
     }
    },

  };
  
</script>
    """

        script_html += f"""

<script>
  const chart{position} = new Chart(
    document.getElementById('{chartId}'),
    config{position}
  );
</script>

        """

        return script_html, chart_html

    def path(self):
        return os.path.join(os.path.dirname(os.path.abspath(__file__)), "js_dist")


# Exploring London Weather Data with Mercury

In [3]:
"""
import numpy as np
import pandas as pd
#from bloxs import B

weather=pd.read_csv('heathrowDataFiltered.csv')

import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(10,7))

def plotchart(y,yticks,label,ax, kind):
    xticks= [1,2,3,4,5,6,7,8,9,10,11,12]
    xlabels = ['Jan','Feb','Mar','Apr','May',
                'Jun','Jul','Aug','Sep','Oct','Nov','Dec']

    ax=weather[weather['Year']==year].plot(kind=kind,y = y, x ='Month', 
                                           yticks = yticks, xticks = xticks,
                                           legend = False, title = y, ax=ax)
                                      
    ax.set_xticklabels(xlabels)
    ax.set_ylabel(label)

yticks = [0,5,10,15,20,25,30]
label="Degrees Celsius"
B("123", "Display line chart", points=[1,4,2,3,5,6], color="red")
plotchart('Tmax', yticks, label, axes[0,0], 'line')
plotchart('Tmin', yticks, label, axes[0,1], 'line')

yticks = [0,10,20,30,40,50,60,70,80,90,100]
label="Millimetres"
plotchart('Rain', yticks, label, axes[1,0], 'bar')

yticks = [0,50,100,150,200,250,300]
label="Hours"
plotchart('Sun', yticks, label, axes[1,1], 'bar')

print(f"Year: {year}")
plt.tight_layout()
"""

'\nimport numpy as np\nimport pandas as pd\n#from bloxs import B\n\nweather=pd.read_csv(\'heathrowDataFiltered.csv\')\n\nimport matplotlib.pyplot as plt\n\nfig, axes = plt.subplots(nrows=2, ncols=2,figsize=(10,7))\n\ndef plotchart(y,yticks,label,ax, kind):\n    xticks= [1,2,3,4,5,6,7,8,9,10,11,12]\n    xlabels = [\'Jan\',\'Feb\',\'Mar\',\'Apr\',\'May\',\n                \'Jun\',\'Jul\',\'Aug\',\'Sep\',\'Oct\',\'Nov\',\'Dec\']\n\n    ax=weather[weather[\'Year\']==year].plot(kind=kind,y = y, x =\'Month\', \n                                           yticks = yticks, xticks = xticks,\n                                           legend = False, title = y, ax=ax)\n                                      \n    ax.set_xticklabels(xlabels)\n    ax.set_ylabel(label)\n\nyticks = [0,5,10,15,20,25,30]\nlabel="Degrees Celsius"\nB("123", "Display line chart", points=[1,4,2,3,5,6], color="red")\nplotchart(\'Tmax\', yticks, label, axes[0,0], \'line\')\nplotchart(\'Tmin\', yticks, label, axes[0,1], \'line

In [4]:
#! pip install bloxs

In [10]:
from bloxs import B
B("123", "Display line chart", points=[1,4,2,3,5,6], color="red")

In [6]:
! python --version

Python 3.10.2
